In [1]:
import numpy as np, cv2

In [2]:
def draw_histo(hist, size=(200, 256)):
    hist_img = np.full((size[0], size[1]), 255, dtype=np.uint8)
    hist = cv2.normalize(hist, None, 0, size[0], cv2.NORM_MINMAX).flatten()

    bin_width = int(np.round(size[1] / hist.shape[0]))
    for i in range(hist.shape[0]):
        x = i * bin_width
        y = size[0] - int(hist[i])
        cv2.rectangle(hist_img, (x, y), (x + bin_width - 1, size[0]), 0, -1)
    return hist_img

image = cv2.imread("images/equalize.jpg", cv2.IMREAD_GRAYSCALE) # 영상 읽기
if image is None: raise Exception("영상 파일 읽기 오류")

bins, ranges = [256], [0, 256]
hist = cv2.calcHist([image], [0], None, bins, ranges)    # 히스토그램 계산

# 히스토그램 누적합 계산
accum_hist = np.zeros(hist.shape[:2], np.float32)
accum_hist[0] = hist[0]
for i in range(1, hist.shape[0]):
    accum_hist[i] = accum_hist[i - 1] + hist[i]

accum_hist = (accum_hist / sum(hist)) * 255                 # 누적합의 정규화
dst1 = [[accum_hist[val] for val in row] for row in image] # 화소값 할당
dst1 = np.array(dst1, np.uint8)

# numpy 함수 사용
#accum_hist = np.cumsum(hist)                      # 누적합 계산
#cv2.normalize(accum_hist, accum_hist, 0, 255, cv2.NORM_MINMAX)  # 정규화

dst1 = cv2.LUT(image, accum_hist.astype("uint8"))  #룩업 테이블로 화소값할당
# dit1 = image[accum_hist.astype('uint8')]         # 파이썬 방식 록업테이블
dst2 = cv2.equalizeHist(image)                # OpenCV 히스토그램 평활화

hist1 = cv2.calcHist([dst1], [0], None, bins, ranges)   # 히스토그램 계산
hist2 = cv2.calcHist([dst2], [0], None, bins, ranges)   # 히스토그램 계산
hist_img = draw_histo(hist)
hist_img1 = draw_histo(hist1)
hist_img2 = draw_histo(hist2)

cv2.imshow("image", image);             cv2.imshow("hist_img", hist_img)
cv2.imshow("dst1_User", dst1);          cv2.imshow("User_hist", hist_img1)
cv2.imshow("dst2_OpenCV", dst2);        cv2.imshow("OpenCV_hist", hist_img2)
cv2.waitKey(0)

-1